# Emsions air établissements 20K - 50K


Ce notebook sert à générer la table des émisions de gaz par des établissements situés dans des communes entre 20k et 50 habitants; 


** donnees **


In [1]:
import pandas as pd  
import numpy as np

In [19]:
import pymysql.cursors

connection = pymysql.connect( host='localhost',
                              user='root',
                              passwd='root',
                              db='energie',
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

In [3]:
gr = pd.read_csv('data/georisques/emissions.csv',sep=',', encoding='utf-8', thousands = ' ', decimal=',', dtype='str')
gr = gr.loc[gr['Milieu'] == 'Air']
gr.head(3)

,Identifiant,Nom_Etablissement,Annee_Emission,Milieu,Polluant,quantite,unite
0,01021035,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Ammoniac (NH3),12500,kg/an
1,01021035,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Protoxyde d'azote (N2O),16800,kg/an
2,01025281,EARL DE PRESSAUGE,2016,Air,Ammoniac (NH3),11100,kg/an


In [4]:
gre = pd.read_csv('data/georisques/etablissements.csv', sep=',', encoding='utf-8', thousands = ' ', decimal=',',
                  dtype='str')
gre.head(3)

,Identifiant,Nom_Etablissement,Numero_SIRET,Adresse,Code_Postal,Commune,Departement,Region,Coordonnees_X,Coordonnees_Y,Code_APE,Libelle_APE,code_eprtr,libelle_eprtr
0,055.00059,Sanders Bretagne,35245494600042,NaN,22200,GUINGAMP,COTES-D'ARMOR,BRETAGNE,194408,2410066,1091Z,Fabrication d'aliments pour animaux de ferme,NaN,NaN
1,055.01776,GUERBET,30849152100081,Rue Denis Papin ZI de Kerpont,56607,LANESTER,MORBIHAN,BRETAGNE,174122,2323310,2110Z,Fabrication de produits pharmaceutiques de base,4.(e),Installations utilisant un procédé chimique ou...
2,057.04930,Poirier,38064754500025,NaN,10200,FONTAINE,AUBE,CHAMPAGNE-ARDENNES,776700,2359885,4312A,Travaux de terrassement courants et travaux pr...,NaN,NaN


In [5]:
emissions = pd.merge(gr, gre, left_on='Nom_Etablissement', right_on='Nom_Etablissement', how='inner')
emissions.head()

,Identifiant_x,Nom_Etablissement,Annee_Emission,Milieu,Polluant,quantite,unite,Identifiant_y,Numero_SIRET,Adresse,Code_Postal,Commune,Departement,Region,Coordonnees_X,Coordonnees_Y,Code_APE,Libelle_APE,code_eprtr,libelle_eprtr
0,01021035,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Ammoniac (NH3),12500,kg/an,01021035,77938855200019,NaN,01480,ARS-SUR-FORMANS,AIN,RHONE-ALPES,792562,2113648,0146Z,Élevage de porcins,NaN,NaN
1,01021035,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Protoxyde d'azote (N2O),16800,kg/an,01021035,77938855200019,NaN,01480,ARS-SUR-FORMANS,AIN,RHONE-ALPES,792562,2113648,0146Z,Élevage de porcins,NaN,NaN
2,01025281,EARL DE PRESSAUGE,2016,Air,Ammoniac (NH3),11100,kg/an,01025281,32527338100013,NaN,01380,BAGE-LA-VILLE,AIN,RHONE-ALPES,800904,2149474,0115Z,Culture du tabac,NaN,NaN
3,01046103,GAEC des Prelions,2016,Air,Ammoniac (NH3),25900,kg/an,01046103,77930225600010,Le Calendras,01290,BIZIAT,AIN,RHONE-ALPES,801148,2138382,0146Z,Élevage de porcins,7.(a).(ii),disposant de 2 000 emplacements pour porcs de ...
4,01050004,EARL Les Cornalières,2016,Air,Ammoniac (NH3),14000,kg/an,01050004,52359360600016,Les Cornalières,01190,BOISSEY,AIN,RHONE-ALPES,804580,2157224,0147Z,Élevage de volailles,7.(a).(i),disposant de 40 000 emplacements pour la volaille


In [6]:
emissions.shape

(5997, 20)

In [7]:
emissions = emissions[emissions['Annee_Emission'] == '2016']
emissions.shape

(5997, 20)

In [8]:
emissionsAir = emissions[emissions['Milieu'] == 'Air']
emissionsAir.shape

(5997, 20)

In [9]:
emissions = emissions[['Nom_Etablissement', 'Annee_Emission', 'Milieu',
       'Polluant', 'quantite', 'unite', 'Code_Postal', 'Libelle_APE', 'libelle_eprtr']]
emissions.head(3)

,Nom_Etablissement,Annee_Emission,Milieu,Polluant,quantite,unite,Code_Postal,Libelle_APE,libelle_eprtr
0,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Ammoniac (NH3),12500,kg/an,01480,Élevage de porcins,NaN
1,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Protoxyde d'azote (N2O),16800,kg/an,01480,Élevage de porcins,NaN
2,EARL DE PRESSAUGE,2016,Air,Ammoniac (NH3),11100,kg/an,01380,Culture du tabac,NaN


## Associer CP et code Insee

In [10]:
# NosDonnées 
# http://www.nosdonnees.fr/wiki/index.php/Fichier:EUCircos_Regions_departements_circonscriptions_communes_gps.csv.gz

url = 'data/nosDonnees/eucircos_regions_departements_circonscriptions_communes_gps.csv'
usecols = ['codes_postaux', 'code_insee']
lalo = pd.read_csv(url, usecols=usecols, sep=";", dtype=str, thousands=' ', decimal=',', encoding='utf-8')
lalo.head(3)

,codes_postaux,code_insee
0,01340,01024
1,01270,01029
2,01370,01038


In [11]:
emissionsInsee = pd.merge(emissions, lalo, how='left', left_on='Code_Postal', right_on='codes_postaux')
emissionsInsee.head(3)

,Nom_Etablissement,Annee_Emission,Milieu,Polluant,quantite,unite,Code_Postal,Libelle_APE,libelle_eprtr,codes_postaux,code_insee
0,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Ammoniac (NH3),12500,kg/an,01480,Élevage de porcins,NaN,01480,01021
1,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Ammoniac (NH3),12500,kg/an,01480,Élevage de porcins,NaN,01480,01030
2,SCEA DOMAINE DU MAS DE VILLENEUVE,2016,Air,Ammoniac (NH3),12500,kg/an,01480,Élevage de porcins,NaN,01480,01075


In [16]:
emissionsInsee = emissionsInsee.fillna(' ')

## to sql

In [20]:
# Insertion BDD
# Il a été nécessaire de changer les NaN en ''

with connection.cursor() as cursor:
    
    sql = ("INSERT INTO `emissions_etablissements` "
           "( `insee`, `etablissement`, `annee`, `milieu`, `polluant`, `quantite`, `unite`, `libelle_ape`, `libelle_eprtr`)"
           "VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s)"
          )

    for i, r in emissionsInsee.iterrows():
        cursor.execute(sql, 
            (r['code_insee'], r['Nom_Etablissement'], r['Annee_Emission'], r['Milieu'], r['Polluant'], 
             r['quantite'], r['unite'], r['Libelle_APE'], r['libelle_eprtr'] )
        )

connection.commit() 